# Langchain - Semantic Search

Este post vamos a ver cómo usar [El cargador de documentos](https://python.langchain.com/docs/concepts/document_loaders), el uso y cómo crear [embeddings](https://python.langchain.com/docs/concepts/embedding_models/) y el uso de [bases de datos vectoriales](https://python.langchain.com/docs/concepts/vectorstores).

Estas funciones están diseñadas para gestionar la recuperación de datos de bases de datos (vector) y otras fuentes para la integración con flujos de trabajo con LLMs.

Son importantes para las aplicaciones que obtienen datos para ser razonados como parte de la inferencia del modelo, como en el caso de [RAG](https://python.langchain.com/docs/concepts/rag/).

Vamos a construir un motor de búsqueda sobre un documento PDF. Esto nos permitirá recuperar datos en el PDF

## Instalación de librerías

Para este post vamos a necesitar instalar las librerías `langchain-community` y `pypdf`, para ello las instalamos con conda

``` bash
conda install langchain-community pypdf -c conda-forge
```

o con pip

``` bash
pip install langchain-community pypdf
```

### Token de LangSmith (opcional)

Mediante LangSmith podemos guardar las llamadas a los modelos y ver las métricas de las llamadas. Pera ello, necesitamos crear un token de LangSmith.

Ahora configuramos LangSmith para que guarde las llamadas a los modelos.

In [1]:
import os
import dotenv

dotenv.load_dotenv()

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")

In [2]:
import os

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")

os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_TRACING"] = LANGSMITH_TRACING

## Documentos y cargadores de documentos

LangChain tiene la librería [Documento](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html), que pretende representar una unidad de texto y metadatos asociados. Tiene tres atributos:

 + ``page_content``: una cadena que representa el contenido;
 + ``metadata``: un dict que contiene metadatos arbitrarios;
 + ``id``: ``(opcional)`` un identificador de cadena para el documento.

El atributo metadata puede capturar información sobre la fuente del documento, su relación con otros documentos y otra información. Un objeto ``Document`` a menudo representa una parte de un documento más grande.

Vamos a ver cómo crear un ``Document``

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## Document loaders

LangChain implementa la herramienta de [cargadores de documentos](https://python.langchain.com/docs/concepts/document_loaders/) que se puede usar con cientos de [fuentes comunes](https://python.langchain.com/docs/integrations/document_loaders/). Esto facilita la incorporación de datos de estas fuentes en su aplicación de IA

Vamos a cargar un PDF como un objeto ``Document``. [Aquí](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data) hay un ejemplo de un PDF, un reporte aunal del 2023 de Nike.

Podemos ver los posibles [cargadores de documentos PDF](https://python.langchain.com/docs/integrations/document_loaders/#pdfs) disponibles en Langchain. Vamos a usar ``PyPDFLoader``, que es bastante ligero.

Primero cargamos el PDF

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

PyPDFLoader carga el contenido de cada hoja del PDF como un ``Document``, así que vemos cuantos documentos tenemos, que debería ser igual al número de hojas del PDF.

In [5]:
print(len(docs))

107


Obtenemos 107 documentos, que son las 107 hojas del PDF.

Vamos a ver el contenido de la primera página

In [6]:
print(docs[0].page_content[:500])

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in it


Vemos también sus metadatos

In [7]:
docs[0].metadata

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creator': 'EDGAR Filing HTML Converter',
 'creationdate': '2023-07-20T16:22:00-04:00',
 'title': '0000320187-23-000039',
 'author': 'EDGAR Online, a division of Donnelley Financial Solutions',
 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31',
 'keywords': '0000320187-23-000039; ; 10-K',
 'moddate': '2023-07-20T16:22:08-04:00',
 'source': 'nke-10k-2023.pdf',
 'total_pages': 107,
 'page': 0,
 'page_label': '1'}

## División del contenido en chunks

Para fines de recuperación de información, una página puede ser una representación demasiado grande. Así que el objetivo al final será recuperar el objeto ``Document`` que responde a una consulta de entrada y dividirlo aún más ayuda a garantizar que los significados de las partes relevantes del documento no se diluyan con el resto del documento.

Podemos usar [text splitters](https://python.langchain.com/docs/concepts/text_splitters) para este propósito.

En este post vamos a usar un divisor de texto simple que particiona en función de los caracteres. Dividiremos el documento en trozos de 1000 caracteres con 200 caracteres de superposición (``overlap``) entre los trozos. La superposición ayuda mitigar la posibilidad de separar una parte de texto con contexto que al separarlo deja de tener el mismo significado.

Vamos a usar [RecursivoCaracterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter), que dividirá recursivamente el documento usando separadores comunes como nuevas líneas hasta que cada fragmento sea del tamaño apropiado. Este es el divisor de texto recomendado para casos de uso de texto genérico.

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

Una vez dividido el documento, vemos el número de chunks

In [9]:
len(all_splits)

516

## Embeddings

Los [embeddings](https://python.langchain.com/docs/concepts/embedding_models) son una forma común de almacenar y buscar datos no estructurados (como texto no estructurado). La idea es crear embeddings de cada chunck y almacenarlos en una base de datos vectorial.

Dada una consulta, la convertimos a embeddings y la comparamos con los embeddings de cada chunk para encontrar el más similar, por ejemplo, usando la similitud de coseno.

``LangChain`` admite embeddings de docenas de [proveedores](https://python.langchain.com/docs/integrations/text_embedding). Estos modelos especifican cómo el texto debe convertirse en un embedding.

Nosotros vamos a usar los modelos de embeddings de Hugging Face. Así que creamos el modelo de embeddings

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Creamos los embeddings de dos chunks

In [12]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

Primero comprobamos que la longitud de los dos embeddings es la misma

In [13]:
assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")

Generated vectors of length 768



## Bases de datos vectoriales

Los objetos [VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) de ``LangChain`` contienen métodos para agregar texto y objetos ``Document`` a la base de datos vectorial y consultarlos utilizando varias métricas de similitud. A menudo se inicializan con modelos de [embeddings](https://python.langchain.com/docs/how_to/embed_text), que determinan cómo se traducen los datos de texto a vectores de embeddings.

``LangChain`` incluye un conjunto de [integraciones](https://python.langchain.com/docs/integrations/vectorstores) con diferentes tecnologías de almacenamiento de embeddings.

 + Algunas base de datos vectoriales están alojadas por un proveedor (por ejemplo, varios proveedores de la nube) y requieren credenciales específicas para usar
 + Algunos (como [Postgres](https://python.langchain.com/docs/integrations/vectorstores/pgvector)) pueden ejecutarse en una infraestructura separada que se puede ejecutar localmente o a través de un tercero
 + Otros pueden ejecutarse en memoria para cargas de trabajo livianas.

En este post vamos a usar [ChromaDB](https://www.maximofn.com/chromadb), así que para ello primero tenemos que instalar la librería de ChromaDB de Langchain

``` bash
pip install -qU langchain-chroma
```

Una vez instalado, podemos crear una base de datos vectorial en memoria

In [14]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

Una vez creada la base de datos vectorial, podemos agregar los chunks a la base de datos

In [15]:
ids = vector_store.add_documents(documents=all_splits)

Una vez que hayamos instanciado un ``VectorStore`` que contiene documentos, podemos consultarlo. ``VectorStore`` incluye métodos para consultar:
 * Sincrónica y asincrónicamente;
 * Por consulta de cadena y por vector;
 * Con y sin devolver puntuaciones de similitud;
 * Por similitud y [máxima relevancia marginal](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search) (para equilibrar la similitud con la consulta a la diversidad en los resultados recuperados).

Los métodos generalmente incluirán una lista de Documento objetos en sus salidas.

### Búsqueda por similitud

Los embeddings típicamente representan el texto como un vector de tal manera que los textos con significados similares están geométricamente cerca. Esto nos permite recuperar información relevante con solo pasar una pregunta, sin conocimiento de ningún término clave específico utilizado en el documento.

De esta manera, podemos recuperar documentos basados en la similitud con una consulta de texto

In [16]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'author': 'EDGAR Online, a division of Donnelley Fi

Sin embargo como esto consume un tiempo de crear el embedding de la consulta y después calcular la similitud con cada chunk, podemos hacer esta consulta de manera asíncrona

In [17]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-

### Búsqueda por similitud con puntuación

Podemos obtener los resultados con las puntuaciones de similitud

In [18]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")

doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.3725222051143646

page_content='Table of Contents
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa 13,418 12,479 11,456 
Greater China 7,248 7,547 8,290 
Asia Pacific & Latin America 6,431 5,955 5,343 
Global Brand Divisions 58 102 25 
Total NIKE Brand 48,763 44,436 42,293 
Converse 2,427 2,346 2,205 
Corporate 27 (72) 40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa 3,531 3,293 2,435 
Greater China 2,283 2,365 3,243 
Asia Pacific & Latin America 1,932 1,896 1,530 
Global Brand Divisions (4,841) (4,262) (3,656)
Converse 676 669 543 
Corporate (2,840) (2,219) (2,261)
Interest expense (income), net (6) 205 262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europe, Middle East & Africa 21

### Devolución del ``Document`` ante una consulta

Si queremos obtener el ``Document`` ante una consulta

In [19]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

## ``Retrievers``

Los objetos ``VectorStore`` no tienen una subclase ``Runnable``. De modo Langchain ofrece el objeto [Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers), que son objetos ``Runnable``, por lo que implementan un conjunto estándar de métodos (por ejemplo, síncronos y asíncronos como ``invoke`` y ``batch``). Aunque podemos construir retrievers a partir de bases de datos vectoriales, los retrievers también pueden interactuar con fuentes de datos que no sean de bases de datos vectoriales (como API externas).

Las ``Vectorstores`` implementan un método ``as_retriever`` que generará un objeto ``Retriever``, específicamente un [VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html).

Estos retrievers incluyen las variables ``search_type`` y ``search_kwargs`` que identifican qué métodos del almacén de vectores subyacente llamar y cómo parametrizarlos.

In [20]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

Y ahora podemos usar el retriever como un ``Runnable`` para recuperar documentos

In [21]:
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[[Document(id='8e65328d-73e5-49be-bd9c-5324dab3f32a', metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-20T16:22:00-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'page': 26, 'page_label': '27', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': 'nke-10k-2023.pdf', 'start_index': 804, 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'title': '0000320187-23-000039', 'total_pages': 107}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

``VectorStoreRetriever`` admite tipos de búsqueda ``similarity`` (predeterminado), ``mmr`` (relevancia marginal máxima), y ``similarity_score_threshold``. Podemos usar este último para limitar la salida de documentos por parte del recuperador por puntuación de similitud.

Podemos crear una versión simple de esto nosotros mismos, sin subclases ``Retriever``. Si elegimos qué método deseamos utilizar para recuperar documentos, podemos crear un ``Runnable`` fácilmente. A continuación construiremos uno alrededor del método ``similarity_search``

In [22]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

Volvemos a usar el retriever para recuperar documentos

In [23]:
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='8e65328d-73e5-49be-bd9c-5324dab3f32a', metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-20T16:22:00-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'page': 26, 'page_label': '27', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': 'nke-10k-2023.pdf', 'start_index': 804, 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'title': '0000320187-23-000039', 'total_pages': 107}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

Las estrategias de recuperación pueden ser ricas y complejas. Por ejemplo:

 * Podemos inferir [reglas y filtros](https://python.langchain.com/docs/how_to/self_query) de una consulta (por ejemplo, "utilizando documentos publicados después de 2020")
 * Podemos [devolver documentos](https://python.langchain.com/docs/how_to/parent_document_retriever) que estén vinculados al contexto recuperado de alguna manera (por ejemplo, a través de alguna taxonomía de documentos)
 * Podemos generar [múltiples embeddings](https://python.langchain.com/docs/how_to/multi_vector/) para cada unidad de contexto
 * Podemos generar [resultados del conjunto](https://python.langchain.com/docs/how_to/ensemble_retriever/) de múltiples retrievers
 * Podemos asignar pesos a documentos, por ejemplo, para pesar [documentos recientes](https://python.langchain.com/docs/how_to/time_weighted_vectorstore) más alto.

En [retrievers](https://python.langchain.com/docs/how_to/#retrievers) encontramos las guías prácticas que cubren cubre estas y otras estrategias de recuperación integradas

También es sencillo extender la clase [BaseRetriever](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html) para implementar [retrievers personalizados](https://python.langchain.com/docs/how_to/custom_retriever).